In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Peptic_ulcer_disease"
cohort = "GSE43580"

# Input paths
in_trait_dir = "../../input/GEO/Peptic_ulcer_disease"
in_cohort_dir = "../../input/GEO/Peptic_ulcer_disease/GSE43580"

# Output paths
out_data_file = "../../output/preprocess/Peptic_ulcer_disease/GSE43580.csv"
out_gene_data_file = "../../output/preprocess/Peptic_ulcer_disease/gene_data/GSE43580.csv"
out_clinical_data_file = "../../output/preprocess/Peptic_ulcer_disease/clinical_data/GSE43580.csv"
json_path = "../../output/preprocess/Peptic_ulcer_disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"SBV - Gene Expression Profiles of Lung Cancer Tumors - Adenocarcinomas and Squamous Cell Carcinomas"
!Series_summary	"This dataset encompassing the profiles of 150 lung cancer tumors was developed to serve as test dataset in the SBV IMPROVER Diagnostic Signature Challenge (sbvimprover.com). The aim of this subchallenge was to verify that it is possible to extract a robust diagnostic signature from gene expression data that can identify stages of different types of lung cancer. Participants were asked to develop and submit a classifier that can stratify lung cancer patients in one of four groups – Stage 1 of Adenocarcinoma (AC Stage 1), Stage 2 of Adenocarcinoma (AC Stage 2), Stage 1 of Squamous cell carcinoma (SCC Stage 1) or Stage 2 of Squamous cell carcinoma (SCC Stage 2). The classifier could be built by using any publicly available gene expression data with related histopathological information and was tested on the independent dataset describ

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Dict, Any, Callable

# 1. Gene Expression Data Availability 
# Based on the background information, this appears to be a gene expression dataset for lung cancer
# The series title and summary suggest gene expression profiles of lung cancer tumors
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait - Peptic ulcer disease
# This dataset is about lung cancer, not peptic ulcer disease
# While there are mentions of "Duodenal ulcer", "Gastric ulcer", etc., these are comorbidities and not the primary focus
trait_row = None  # Peptic ulcer disease trait data is not available

# For age - available at row 1, but not needed since trait is not available
age_row = 1

# For gender - available at row 0, but not needed since trait is not available
gender_row = 0

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """
    Convert clinical diagnosis text to binary indicator for peptic ulcer disease.
    1 = has peptic ulcer disease, 0 = does not have peptic ulcer disease, None = unknown
    """
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check for various types of ulcers related to peptic ulcer disease
    peptic_ulcer_keywords = ["duodenal ulcer", "gastric ulcer", "stomach ulcer", "peptic ulcer"]
    
    # Convert to lowercase for case-insensitive matching
    value_lower = value.lower()
    
    # Check if any peptic ulcer related keyword is present
    has_peptic_ulcer = any(keyword in value_lower for keyword in peptic_ulcer_keywords)
    
    return 1 if has_peptic_ulcer else 0

def convert_age(value):
    """
    Convert age string to numeric value.
    Return None for unknown or NA values.
    """
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if value is NA
    if value == 'NA':
        return None
    
    try:
        # Convert to integer
        return int(value)
    except ValueError:
        return None

def convert_gender(value):
    """
    Convert gender string to binary (0 = female, 1 = male).
    Return None for unknown values.
    """
    if pd.isna(value):
        return None
    
    # Extract value after colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if value == 'female':
        return 0
    elif value == 'male':
        return 1
    else:
        return None

# 3. Save Metadata
# The trait data is available if trait_row is not None
is_trait_available = trait_row is not None

# Conduct initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step since trait_row is None (peptic ulcer disease data is not available)
if trait_row is not None:
    # This code block won't execute because trait_row is None
    # Clinical feature extraction would go here if trait data were available
    pass


A new JSON file was created at: ../../output/preprocess/Peptic_ulcer_disease/cohort_info.json


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be Affymetrix probe IDs (like "1007_s_at") rather than 
# standard human gene symbols (which would look like BRCA1, TP53, etc.)
# Affymetrix probe IDs need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns in gene_annotation for mapping
# From the gene_annotation preview, we can see:
# - 'ID' column contains probe IDs like 1007_s_at, which match the gene expression data index
# - 'Gene Symbol' column contains the human gene symbols we need like DDR1, RFC2, etc.

# 2. Get gene mapping dataframe by extracting the ID and Gene Symbol columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first few gene symbols to verify the mapping worked
print("First 10 gene symbols after mapping:")
print(gene_data.index[:10])


First 10 gene symbols after mapping:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since we determined in Step 2 that trait_row is None (no peptic ulcer disease data)
# and is_trait_available is False, we can't proceed with proper clinical data linking
# Instead, we need to create dummy clinical data with a constant trait value

# Create minimal clinical dataframe with samples matching gene data
sample_ids = normalized_gene_data.columns
dummy_clinical_df = pd.DataFrame(index=[trait], 
                                  columns=sample_ids,
                                  data=[[1] * len(sample_ids)])  # All samples marked as having the trait

print(f"Created dummy clinical data with shape: {dummy_clinical_df.shape}")

# 2. Link the clinical and genetic data (even though clinical data is just a constant)
linked_data = geo_link_clinical_genetic_data(dummy_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# Identify trait column
trait_col = linked_data.columns[0]
print(f"Using trait column: {trait_col}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait_col)
print(f"Shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
# Since we created a dummy trait with all 1s, this trait is severely biased
is_trait_biased = True
print(f"The trait {trait} is severely biased (all samples have the same value).")
unbiased_linked_data = linked_data

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=False,  # We previously determined peptic ulcer disease data is not available
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data for lung cancer samples but lacks {trait} information."
)

# 6. Since the data is not usable (no trait variation), don't save it
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. Dataset lacks necessary {trait} variation for association studies.")

Normalized gene data saved to ../../output/preprocess/Peptic_ulcer_disease/gene_data/GSE43580.csv
Created dummy clinical data with shape: (1, 150)
Linked data shape: (150, 19846)
Using trait column: Peptic_ulcer_disease


Shape after handling missing values: (150, 19846)
The trait Peptic_ulcer_disease is severely biased (all samples have the same value).
Data quality check failed. Dataset lacks necessary Peptic_ulcer_disease variation for association studies.
